# Laphet (Version 0.7) with Myanmar Names Dataset
## Using nn.Embedding   

ဒီ Jupyter notebook က Laphet LM Toolkit ကို သုံးပြီး MLP, Bi-LSTM, Transformer, BERT, GPT အခြေခံတဲ့ language model (LM) တွေကို ဆောက်ကြည့်တာ၊ ဆောက်ထားပြီးသား LM ကိုသုံးပြီး text generation စမ်းလုပ်ပြထားတာပါ။  



## Dataset Information

Dataset link: [https://github.com/ye-kyaw-thu/myRoman](https://github.com/ye-kyaw-thu/myRoman)  

myRoman ဒေတာ format က အောက်ပါအတိုင်း ရှိတာမို့  

1	ကကြိုးကြာ	က ကြိုး ကြာ	ka kyoe kyar  
2	ကချောင်လွမ်းနန်း	က ချောင် လွမ်း နန်း	ka chaung lwan nan  
3	ကချောင်လွမ်းနန်း	က ချောင် လွမ်း နန်း	ka chaung lwan nang  

ဝဏ္ဏဖြတ်ထားတဲ့ (i.e. syllable) တတိယ ကော်လံကို ယူသုံးထားပါတယ်။  
ပြီးတော့ ‌ZWNJ (U+200C) နဲ့ SHY (U+00AD) သင်္ကေတတွေကို clean လုပ်ပြီး သုံးခဲ့ပါတယ်။  

In [1]:
ls ./data/myRoman/

dev_name.txt  start_names.txt  test_name.txt  train_name.txt


train_name.txt ဖိုင်က training လုပ်တဲ့အခါမှာ သုံးဖို့အတွက် ပြင်ထားတာ။  
dev_name.txt ဖိုင်က development သို့မဟုတ် validation အတွက် သုံးဖို့ ပြင်ထားတာ။  
test_name.txt ဖိုင်က testing/evaluation လုပ်ဖို့အတွက် ပြင်ထားတာ။  
start_names.txt ဖိုင်က ဆောက်ထားတဲ့ language model တွေနဲ့ မြန်မာနာမည်တွေကို generate လုပ်ခိုင်းတဲ့အခါမှာ ကိုယ်က စစေချင်တဲ့ ဝဏ္ဏ (i.e. syllable) စာလုံးတွေကို ပြင်ထားတဲ့ ဖိုင်ပါ။  

In [8]:
!wc ./data/myRoman/*.txt

   1000    3193   37204 ./data/myRoman/dev_name.txt
     15      18     205 ./data/myRoman/start_names.txt
   1000    3246   37728 ./data/myRoman/test_name.txt
  27246   88007 1028564 ./data/myRoman/train_name.txt
  29261   94464 1103701 total


အထက်မှာ မြင်ရတဲ့အတိုင်း training data က စာကြောင်းရေ နှစ်သောင်းခုနှစ်ထောင်ကျော်ရှိပါတယ်။  
development နဲ့ test ဒေတာကိုတော့ စာကြောင်းရေ တစ်ထောင်စီ ထားထားပါတယ်။

File format က sentence တစ်ကြောင်းကို နာမည်တစ်ခုမို့လို့ တနည်းအားဖြင့် training ဒေတာက မြန်မာစာနဲ့ ရိုက်ထားတဲ့ နာမည်ပေါင်း နှစ်သောင်းခုနှစ်ထောင်ကျော်ရှိပါတယ်။  

In [10]:
!head ./data/myRoman/train_name.txt

သူ ရ ဖြိုး မင်း
ခန့် မင်း သူ
ခိုင် ဇာ ဝင်း
ကြည် အောင် မင်း
အောင် ဇင် ခန့်
ကျော် လင်း ဝေ
ခတ္တာ ဇော်
ကောင်း ခန့် လင်း
အောင် ထူး
အောင် သန်း ထွန်း


In [11]:
!head ./data/myRoman/dev_name.txt

ကံ ကောင်း
လဲ့ လဲ့
ကျော် တိုး ဝေ
သက် ဘုန်း နိုင်
သိမ့် သူ သူ ကျော်
သဲ အိန္ဒြေ အောင်
ကျော် သန်း ထွန်း
အိ မွန် သန့်
အေး မြင့် မိုရ် ပိုင်
အောင် ဖုန်း ပိုင်


In [12]:
!head ./data/myRoman/test_name.txt

အေး သိင်္ဂီ
သာ စော
ကျော့ ယ မင်း သွယ်
မိုး ဆု ပန်
ခေါ် ထန့်
ခိုင် ခိုင် ဖြိုး
ကြယ် စင် မင်း ခန့်
သေး မယ်
သိဒ္ဓိ ဆန်း
က လျာ သန်း


In [13]:
!head ./data/myRoman/start_names.txt

ကျော်
မ မ
အေး
လှ လှ

မြ အေး
သ
မောင်
မြင့် မြင့်
ရွှေ


start_names.txt ဖိုင်ထဲမှာ blank line ကိုလည်း တမင်တကာ ထားပြီး စမ်းထားတာပါ။ ကျပန်း အစစာလုံးနဲ့ ပရိုဂရမ်က name generate လုပ်ပေးသလား စမ်းချင်လို့ပါ။  

## --help of Laphet (Version 0.7)

In [66]:
!python ./laphet.py --help

usage: laphet.py [-h] --model_type {mlp,bilstm,transformer,bert,gpt} [--train]
                 [--generate] [--test] [--data DATA] [--model MODEL]
                 [--vocab VOCAB] [--dev_file DEV_FILE] [--test_file TEST_FILE]
                 [--prompt PROMPT] [--input INPUT] [--seq_len SEQ_LEN]
                 [--output OUTPUT] [--no_of_generation NO_OF_GENERATION]
                 [--epochs EPOCHS] [--batch_size BATCH_SIZE] [--lr LR]
                 [--embed_dim EMBED_DIM] [--num_heads NUM_HEADS]
                 [--num_layers NUM_LAYERS] [--hidden_dim HIDDEN_DIM]
                 [--ff_dim FF_DIM] [--dropout DROPOUT]
                 [--temperature TEMPERATURE] [--top_k TOP_K] [--top_p TOP_P]
                 [--embedding_method {nn.Embedding,fasttext_freeze,fasttext_no_freeze}]
                 [--fasttext_model FASTTEXT_MODEL]

Laphet language model toolkit for Burmese.

options:
  -h, --help            show this help message and exit
  --model_type {mlp,bilstm,transformer,bert

ကိုယ်စက်ထဲမှာ pytorch အပါအဝင် လိုအပ်တဲ့ python library တွေကို မှန်မှန်ကန်ကန် install လုပ်ထားရင် အထက်ပါလိုမျိုး help screen ကို မြင်ရပါလိမ့်မယ်။   

Version 0.7 မှာ အသစ်တိုးထားတဲ့ command line arguments တွေက  
- --embedding_method
- --fasttext_model

Version 0.6 မှာတုန်းက simple embedding method ဖြစ်တဲ့ nn.Embedding ကိုပဲသုံးထားပြီ။ ခု version 0.7 မှာတော့ fasttext embedding feature နဲ့ run လို့ ရသွားပါပြီ။ အဲဒီမှာတင် freeze နဲ့ no_freeze (feature finetuning) ဆိုပြီး နှစ်မျိုး ထပ်ကွဲသွားပါလိမ့်မယ်။ ဒီ notebook မှာတော့ nn.Embedding နဲ့ပဲ run ပြပါမယ်။  

## Multilayer Perceptron (MLP) based Language Modeling

အရင်ဆုံး MLP-based LM ကို ဆောက်မယ်။ ပြီးရင် text generation လုပ်ကြည့်မယ်။ ပြီးရင် test data ကိုသုံးပြီး evaluation လုပ်ကြည့်မယ်။  

## Training

In [67]:
!time python laphet.py --model_type mlp --train --data ./data/myRoman/train_name.txt \
--dev_file ./data/myRoman/dev_name.txt --model ./model/name/mlp.model \
--seq_len 50 --epochs 10 --batch_size 32 --lr 0.0001 \
--embedding_method nn.Embedding

Epoch 1/10 (Training): 100%|█████████████████| 852/852 [00:06<00:00, 128.90it/s]
Epoch 1, Training Loss: 1.3342
Epoch 1/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 508.30it/s]
Epoch 1, Validation Loss: 1.0547
Best model saved at ./model/name/mlp.model with validation loss: 1.0547
Epoch 2/10 (Training): 100%|█████████████████| 852/852 [00:06<00:00, 133.98it/s]
Epoch 2, Training Loss: 1.0531
Epoch 2/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 497.58it/s]
Epoch 2, Validation Loss: 1.0526
Best model saved at ./model/name/mlp.model with validation loss: 1.0526
Epoch 3/10 (Training): 100%|█████████████████| 852/852 [00:06<00:00, 134.54it/s]
Epoch 3, Training Loss: 1.0522
Epoch 3/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 514.45it/s]
Epoch 3, Validation Loss: 1.0522
Best model saved at ./model/name/mlp.model with validation loss: 1.0522
Epoch 4/10 (Training): 100%|█████████████████| 852/852 [00:06<00:00, 133.04it/s]
Epoch 4, Training Loss: 1

## GPU Usage for MLP LM Building

ထွက်လာတဲ့ vocab ဖိုင်နဲ့ model ဖိုင်ကို လေ့လာကြည့်ရအောင်...

In [71]:
ls -lh ./model/name/mlp.model*

-rw-rw-r-- 1 ye ye 4.8M Jan 28 22:30 ./model/name/mlp.model
-rw-rw-r-- 1 ye ye  24K Jan 28 22:29 ./model/name/mlp.model.vocab


In [72]:
!file ./model/name/mlp.model

./model/name/mlp.model: Zip archive data, at least v0.0 to extract, compression method=store


In [73]:
!head ./model/name/mlp.model.vocab

လက္ခ	0
အား	1
ကျဲရ်	2
ငြိမ့်	3
ဒွတ်	4
နဲမ်	5
ဟင်	6
လွေ	7
ဟေ	8
ချွတ်	9


In [74]:
!tail ./model/name/mlp.model.vocab

ယော်	1433
လင်း	1434
ဂျက်ခ်	1435
လန်း	1436
ကယ်လ်	1437
မိန်း	1438
ဇိန်း	1439
အဉ္ဇ	1440
အန့်	1441
ကုန်	1442


## Text Generation

In [77]:
!time python -u laphet.py --model_type mlp --generate --model ./model/name/mlp.model \
--seq_len 50 --prompt "ရဲ" --no_of_generation 10 \
--embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: ရဲ ဆူ ဟွမ် ကု ဇင်း စီး ခွေး ဇင်း 

လူနာမည်ဆိုရင်တော့ ဝဏ္ဏအလုံး ၅၀ ဆိုတာက မဖြစ်နိုင်လို့ နှစ်လုံးလောက်ပဲ ထားပြီး text generate လုပ်ခိုင်းကြည့်ရအောင်။   
sequence length ကို --seq_len 2 ဆိုတဲ့ setting နဲ့ run မယ်။ ရလဒ်က အောက်ပါအတိုင်းပါ။   
တစ်ခုရှိတာက တစ်ခေါက် generate လုပ်ခိုင်းတိုင်း နာမည်အသစ်တွေကိုပဲ generate လုပ်မှာမို့ run တဲ့အခေါက်တိုင်းမှာ မတူတဲ့ syllable sequence တွေကိုပဲ ထုတ်ပေးပါလိမ့်မယ်။  

In [78]:
!time python -u laphet.py --model_type mlp --generate --model ./model/name/mlp.model \
--seq_len 2 --prompt "ရဲ" --no_of_generation 10 \
--embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: ရဲ ဿန်း ဆန့်
Generated Text 2: ရဲ

myRoman ရဲ့ ဒေတာမှာက တိုင်းရင်းသားနာမည်မျိုးစုံ ပါဝင်ပါတယ်။ အဲဒါကြောင့် ဗမာနာမည်အနေနဲ့ ကြည့်ရင် ထူးဆန်းတဲ့ နာမည်မျိုးတွေလည်း ထုတ်ပေးပါလိမ့်မယ်။ သေချာတာက MLP-based language model က အလုပ်လုပ်ပေးတာကိုတော့ တွေ့ရပါလိမ့်မယ်။  

## Text Generation with Start-Word Input File

အင်္ဂလိပ်လိုခေါင်းစဉ်မှာက start word လို့ပေးထားပေမဲ့ ကျွန်တော်တို့ လက်ရှိ training လုပ်ထားတာက syllable သို့မဟုတ် ဝဏ္ဏဖြတ်ပြီး လုပ်ထားတာမို့လို့ start word သတ်မှတ်ပေးထားတဲ့ ဖိုင်မှာလည်း syllable unit ဖြတ်ပေးထားမှ အဆင်ပြေပါလိမ့်မယ်။ word ဖြတ်ပြီး training လုပ်ထားတဲ့ မော်ဒယ်အတွက်ဆိုရင်တော့ word unit နဲ့ သွားပါ။  

In [79]:
!time python laphet.py --model_type mlp --generate --model ./model/name/mlp.model \
--seq_len 2 --input ./data/myRoman/start_names.txt --no_of_generation 5 \
--output ./output/name/mlp_gen_texts.txt --embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Random Prompt Generated: ဂုံ
Generated texts saved 

ထွက်လာတဲ့ output ဖိုင်ကို လေ့လာကြည့်ရအောင်။  
--no_of_generation 5 ဆိုပြီး ထားခဲ့တာမို့လို့ input file ထဲက အစ စာလုံး တစ်ခုစီအတွက် ဥပမာ ငါးခုစီ generate လုပ်ပေးပါလိမ့်မယ်။  

In [81]:
!cat ./output/name/mlp_gen_texts.txt

ကျော် ပြန့် မန်
ကျော် ဂျား ဟူး
ကျော် ဩ ကိုက်
ကျော် ဆိုက် ကြံ့
ကျော် ထိုက် ချုံး
မ မ ဆဲ တောင်း
မ မ အိန္ဒြ မြေး
မ မ ပုံ့ မိန်း
မ မ ဟို့စ် ဗေး
မ မ အဲန် ရှာမ်
အေး ညှာ မြိုင်
အေး တောင် တွယ်
အေး ဇ ဆွင်
အေး လျှံ ပျို
အေး ရှိုင်းန်း ရှန်း
လှ လှ ဆုမ်း လင်္ကာ
လှ လှ ဘယ်လ် ဂျက်ခ်
လှ လှ သန္တာ မျှား
လှ လှ ထား ကျင့်
လှ လှ ချင်း ခဂ္ဂါ
ဂုံ ယု ဣန္တာ
ဂုံ စိမ်း တိုး
ဂုံ ညှင်း စုက္က
ဂုံ စိုး စင်္ကြာ
ဂုံ နောင့် ရှီ
မြ အေး ဆွေ ဂိုး
မြ အေး ယံ အား
မြ အေး မျှော် ကီး
မြ အေး ဒို အိန္ဒြ
မြ အေး မြူ ဇမ်
သ ထောင် စင်္ကြာ
သ ယဉ်း ဒါ
သ ဋေ ရှိုင်းန်း
သ လွမ်း ငုံ
သ အဏ္ဏ ရီ
မောင် ဟို အောန်
မောင် ပွင့်် အစ်
မောင် အပ္ပ ထွာ
မောင် မင်္ဂ ဇံ
မောင် နဲမ်း အန်ဒ်
မြင့် မြင့် လမ့် မြင့်
မြင့် မြင့် ဋာ ဟို့စ်
မြင့် မြင့် ချို လမ်
မြင့် မြင့် ဏာ မ
မြင့် မြင့် မ ကီး
ရွှေ ဂျေ ဆယ်
ရွှေ ဖား မြူ
ရွှေ တွမ် ပြေ
ရွှေ ရွမ်း ရိန်း
ရွှေ နု ဟွမ်း
အဂ္ဂ မှိုင် ခွန်
အဂ္ဂ ရောင်းန် ဂဲလ်
အဂ္ဂ သိဉ္စည်း သွဲ့
အဂ္ဂ ဂိုး ပြာ
အဂ္ဂ ခေါ ခိုင်
ဥက္ကာ ဘဲလ် သျှမ်း
ဥက္ကာ ဆုံး မီး
ဥက္ကာ ဖျား နတ်
ဥက္ကာ ဖိုက် ဇုန်
ဥက္ကာ တွယ် ကီး
သိင်္ဂီ နွမ် နိုး
သိင်္ဂီ တုန်း ခြင်း
သိင

input ဖိုင်ထဲမှာ ပါတဲ့ စာလုံးတွေကို မေ့နေနိုင်တာမို့ ပြန်ရိုက်ပြရရင် အောက်ပါအတိုင်းပါ။  

In [28]:
!cat ./data/myRoman/start_names.txt

ကျော်
မ မ
အေး
လှ လှ

မြ အေး
သ
မောင်
မြင့် မြင့်
ရွှေ
အဂ္ဂ
ဥက္ကာ
သိင်္ဂီ
မေ
ခိုင်


## Testing and Evaluation

Testing လုပ်ချင်တာမို့လို့ --test ဆိုတဲ့ command option နဲ့ run ရပါလိမ့်မယ်။  
test ဖိုင်ရှိတဲ့ path ကို ညွှန်းပေးရပါလိမ့်မယ်။  
--test_file ./data/myRoman/test_name.txt  

In [82]:
!time python laphet.py --model_type mlp --test --model ./model/name/mlp.model \
--test_file ./data/myRoman/test_name.txt --seq_len 50 --batch_size 64 \
--embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:429: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Testing: 100%|██████████████████████████████████| 1

Laphet LM Toolkit မှာ Perplexity ရော၊ Cross-Entropy နဲ့ရော evaluation လုပ်ကြည့်ထားပါတယ်။  

## Bi-LSTM based Language Modeling

ဒီတစ်ခါတော့ command တစ်ခုစီကို အသေးစိတ်မရှင်းတော့ဘူး။  
--model_type ဆိုတဲ့ နေရာမှာ bilstm ကို setup လုပ်ရင်ရပါပြီ။  
ထွက်လာမယ့် model ဖိုင်နာမည်ကိုတော့ မတူအောင်ပေးသင့်တာပေါ့။  

## Training

In [83]:
!time python laphet.py --model_type bilstm --train --data ./data/myRoman/train_name.txt \
--dev_file ./data/myRoman/dev_name.txt --model ./model/name/bilstm.model --seq_len 50 \
--epochs 10 --batch_size 32 --lr 0.0001 \
--embedding_method nn.Embedding 

Epoch 1/10 (Training): 100%|██████████████████| 852/852 [00:15<00:00, 55.37it/s]
Epoch 1, Training Loss: 0.4839
Epoch 1/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 169.22it/s]
Epoch 1, Validation Loss: 0.3251
Best model saved at ./model/name/bilstm.model with validation loss: 0.3251
Epoch 2/10 (Training): 100%|██████████████████| 852/852 [00:15<00:00, 56.16it/s]
Epoch 2, Training Loss: 0.3135
Epoch 2/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 169.17it/s]
Epoch 2, Validation Loss: 0.3098
Best model saved at ./model/name/bilstm.model with validation loss: 0.3098
Epoch 3/10 (Training): 100%|██████████████████| 852/852 [00:15<00:00, 55.37it/s]
Epoch 3, Training Loss: 0.3058
Epoch 3/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 166.79it/s]
Epoch 3, Validation Loss: 0.3022
Best model saved at ./model/name/bilstm.model with validation loss: 0.3022
Epoch 4/10 (Training): 100%|██████████████████| 852/852 [00:15<00:00, 56.25it/s]
Epoch 4, Trainin

## GPU Usage of Bi-LSTM LM Building

In [84]:
!ls -lh ./model/name/bilstm.model*

-rw-rw-r-- 1 ye ye 92M Jan 28 22:40 ./model/name/bilstm.model
-rw-rw-r-- 1 ye ye 24K Jan 28 22:37 ./model/name/bilstm.model.vocab


## Text Generation

--prompt "ရဲ" ဆိုတဲ့ setting နဲ့ run မှာမို့လို့ "ရဲ" နဲ့ စတဲ့ နာမည်တွေကို generate လုပ်ပေးသွားပါလိမ့်မယ်။  
sequence length က ၅၀ ထားထားတာမို့လို့ နာမည်အရှည်ကြီးတွေ ရိုက်ထုတ်ပေးပါလိမ့်မယ်။  
ပြီးတော့ --no_of_generation ကို ၁၀ ထားထားတာမို့လို့ စုစုပေါင်း စာကြောင်း ၁၀ကြောင်းအနေနဲ့ generate လုပ်ပေးသွားပါလိမ့်မယ်။  

In [86]:
!time python laphet.py --model_type bilstm --generate --model ./model/name/bilstm.model \
--seq_len 50 --prompt "ခင်" --no_of_generation 10 \
--embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: ခင် ထိန် ကူ ကြောင် ပဲန်း လန့် ဖီ 

## Text Generation with Start-Word Input File

In [93]:
!time python laphet.py --model_type bilstm --generate --model ./model/name/bilstm.model \
--seq_len 2 --input ./data/myRoman/start_names.txt --no_of_generation 5 \
--output ./output/name/bilstm_gen_texts.txt --embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
All logits are -Inf after filtering! Fallback to ra

In [94]:
!cat ./output/name/bilstm_gen_texts.txt

ကျော် ဖီ သူ
ကျော် ဖီ သူ
ကျော် ဖီ သူ
ကျော် ဖီ သူ
ကျော် ဖီ သူ
မ မ ပဲန်း ကြောင်
မ မ ပဲန်း ထိန်
မ မ လန့် ကန်း
မ မ ကန်း ကြောင်
မ မ ကန်း ဖီ
အေး ဖီ ထိန်
အေး ထိန် ဖီ
အေး ဖီ ထိန်
အေး ဖီ ကြောင်
အေး ဖီ ဖီ
လှ လှ ပျို ထိန်
လှ လှ လန့် ပဲန်း
လှ လှ ကြောင် ထိန်
လှ လှ ပဲန်း ကန်း
လှ လှ ပျို ပဲန်း
ရော် ကူ ကန်း
ရော် ဖီ ကြောင်
ရော် ပဲန်း ပဲန်း
ရော် ပျို ကန်း
ရော် ဖီ ပျို
မြ အေး ဖီ ထိန်
မြ အေး ထိန် ဖီ
မြ အေး ဖီ ဖီ
မြ အေး ပျို ပျို
မြ အေး ဖီ ဖီ
သ ကန်း ကူ
သ ကြောင် ကူ
သ ထိန် လန့်
သ လန့် ထိန်
သ ဖီ ထိန်
မောင် ကန်း ထိန်
မောင် ဖီ လန့်
မောင် ဖီ ကန်း
မောင် ဖီ ထိန်
မောင် ထိန် ဖီ
မြင့် မြင့် ဖီ ကန်း
မြင့် မြင့် ဖီ ဖီ
မြင့် မြင့် ထိန် ဖီ
မြင့် မြင့် ပျို ပဲန်း
မြင့် မြင့် ထိန် ဖီ
ရွှေ ပျို ပျို
ရွှေ ကြောင် ပဲန်း
ရွှေ ပဲန်း ပဲန်း
ရွှေ ပျို ဖီ
ရွှေ ဖီ ဖီ
အဂ္ဂ ထိန် ထိန်
အဂ္ဂ လန့် ကူ
အဂ္ဂ ဖီ ဖီ
အဂ္ဂ ဖီ ကန်း
အဂ္ဂ ဖီ လန့်
ဥက္ကာ ကူ ကြောင်
ဥက္ကာ ကူ ကြောင်
ဥက္ကာ ကြောင် ထိန်
ဥက္ကာ ထိန် ဖီ
ဥက္ကာ လန့် ကူ
သိင်္ဂီ ပဲန်း ပဲန်း
သိင်္ဂီ ပျို ကန်း
သိင်္ဂီ ကန်း ကူ
သိင်္ဂီ ပဲန်း ကန်း
သိင်္ဂီ ကန်း ဖီ
မေ ဖီ ဖီ
မေ ပဲန်း ကြောင်
မေ ဖီ ကြောင်
မေ

In [95]:
!wc ./output/name/bilstm_gen_texts.txt

  75  245 2756 ./output/name/bilstm_gen_texts.txt


## Testing/Evaluation

In [96]:
!time python laphet.py --model_type bilstm --test --model ./model/name/bilstm.model \
--test_file ./data/myRoman/test_name.txt --seq_len 50 --batch_size 64 \
--embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:429: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Testing: 100%|██████████████████████████████████| 1

So far, so good! :)

# Transformer based Language Modeling 

## Training

In [97]:
!time python laphet.py --model_type transformer --train --data ./data/myRoman/train_name.txt \
--dev_file ./data/myRoman/dev_name.txt --model ./model/name/transformer.model --seq_len 50 \
--epochs 10 --batch_size 32 --lr 0.0001 --embedding_method nn.Embedding

Epoch 1/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 196.83it/s]
Epoch 1, Training Loss: 0.2568
Epoch 1/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 595.16it/s]
Epoch 1, Validation Loss: 0.0440
Best model saved at ./model/name/transformer.model with validation loss: 0.0440
Epoch 2/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 210.87it/s]
Epoch 2, Training Loss: 0.0294
Epoch 2/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 598.45it/s]
Epoch 2, Validation Loss: 0.0212
Best model saved at ./model/name/transformer.model with validation loss: 0.0212
Epoch 3/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 208.29it/s]
Epoch 3, Training Loss: 0.0153
Epoch 3/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 591.43it/s]
Epoch 3, Validation Loss: 0.0135
Best model saved at ./model/name/transformer.model with validation loss: 0.0135
Epoch 4/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 197.80it/s]
E

## GPU Usage of Transformer based LM Building

မော်ဒယ် အမျိုးအစားပေါ်ကို မူတည်ပြီးတော့ training လုပ်တဲ့အခါမှာ ကြာချိန်က တူမှာ မဟုတ်ပါဘူး။ လက်ရှိမှာ နာမည်ဒေတာမို့လို့ တကယ်က စာကြောင်းအနေနဲ့ တိုပါတယ်။ ပြီးတော့ သုံးထားတဲ့ GPU အမျိုးအစားနဲ့ memory အပေါ်မှာလည်း မူတည်ပါလိမ့်မယ်။  

In [98]:
!ls -lh ./model/name/transformer.model*

-rw-rw-r-- 1 ye ye 11M Jan 28 22:45 ./model/name/transformer.model
-rw-rw-r-- 1 ye ye 24K Jan 28 22:45 ./model/name/transformer.model.vocab


## Text Generation

In [99]:
!time python laphet.py --model_type transformer --generate --model ./model/name/transformer.model \
--seq_len 50 --prompt "အိ" --no_of_generation 10 \
--embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: အိ စန္ဒ ဟ ယှဉ် တိုင် အဲလ် အမ္မ အဲ

Transformer based LM ရဲ့ text generation ဥပမာတချို့ကတော့ အထက်မှာ မြင်ရတဲ့ အတိုင်းပါပဲ။  
တကယ်က training ဒေတာက syllable ဖြတ်ထားတဲ့ မြန်မာနာမည်တွေမို့လို့ စာကြောင်းလိုမျိုး မရှည်ပါဘူး။  
တမင်တကာ စာကြောင်းအရှည်ကြီး generate လုပ်ခိုင်းကြည့်တာပါ။   

--seq_len ကို 2 ပဲ ထားပြီး၊ --no_of_generation ကိုလည်း 5 ပဲထားပြီး နောက်တစ်ခေါက် ထပ် run ကြည့်ရအောင်။  

In [100]:
!time python laphet.py --model_type transformer --generate --model ./model/name/transformer.model \
--seq_len 2 --prompt "အိ" --no_of_generation 5 \
--embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: အိ ဗေး ဖွေး
Generated Text 2: အိ 

နာမည်အသစ် generate လုပ်ကြည့်တာ မဆိုးဘူးလို့ ထင်ပါတယ်။  

## Text Generation with Start-Word Input File

In [101]:
!time python laphet.py --model_type transformer --generate --model ./model/name/transformer.model \
--seq_len 2 --input ./data/myRoman/start_names.txt --no_of_generation 5 \
--output ./output/name/transformer_gen_texts.txt --embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Random Prompt Generated: ဟေး
Generated texts saved 

--output option သုံးပြီးတော့ ထွက်လာတဲ့ generated sentence တွေကို ဖိုင်ထဲမှာ သိမ်းခိုင်းထားတာမို့လို့ အဲဒီဖိုင်ကို cat command နဲ့ ရိုက်ထုတ်ပြခိုင်းပြီး ဘယ်လို စာကြောင်းတွေ ထွက်လာသလဲ ဆိုတာကို လေ့လာကြည့်ကြရအောင်။  

In [102]:
!cat ./output/name/transformer_gen_texts.txt

ကျော် သျှန် အိုး
ကျော် ပါယ် ခေါမ်
ကျော် ဆွန်း အဲလ်
ကျော် ဟိမ်း အဲလ်
ကျော် ကြယ် ခ
မ မ ယျ လဲ
မ မ လာန်း အိုက်
မ မ သောင် အဲလ်
မ မ မွန် ယဉ်
မ မ ပါ သွဲ့
အေး လွှမ်း ဘော်
အေး ယန် မြန်
အေး ခြိမ့် တိုင်
အေး တေး အဲလ်
အေး သိဉ္စည်း ဆွင်း
လှ လှ တု မွင်း
လှ လှ ဝိ [UNK]
လှ လှ ရည် ဆောမ်း
လှ လှ စူ မြော်
လှ လှ လက်ျာ ချောင်း
ဟေး ဆိုင်း ဒိုင်
ဟေး ဆိုင်း ငုံ
ဟေး အဲလ် ကတ္တီ
ဟေး ဆိုင်း ခန်း
ဟေး မြန် ထွတ်
မြ အေး စန်း ကျီး
မြ အေး ဓူ အဲလ်
မြ အေး ကံ့ ကောင်း
မြ အေး ဖေ ကုမ္မာ
မြ အေး ခေါ ကျိမ်
သ တယ်လ် မောင်း
သ အိန္မာ ကြိုက်
သ ပြန့် ထဲ
သ တြာ အဲလ်
သ လှန် ဆွင်း
မောင် တည် တက္ခ
မောင် ဝဏ် အဲလ်
မောင် လမ့် ထော
မောင် ရှာမ် နည်
မောင် ကယ်လ် အဲလ်
မြင့် မြင့် ချမ်း မာရ်
မြင့် မြင့် စု ထင်
မြင့် မြင့် ဂိမ် အဲလ်
မြင့် မြင့် ကံ့ ပ
မြင့် မြင့် ထ အိန်
ရွှေ သဒ္ဓါ ဖုန်း
ရွှေ ချေ ရက်
ရွှေ တွန်း အဲလ်
ရွှေ မီးရ် အဲလ်
ရွှေ အမ် မြင့်
အဂ္ဂ ဆုမ်း အဲလ်
အဂ္ဂ ဘုဏ်း ဗန်
အဂ္ဂ တီးလ် ခွမ်း
အဂ္ဂ အို [PAD]
အဂ္ဂ ဟြေ အဲလ်
ဥက္ကာ ဖ ဘွား
ဥက္ကာ ဒါ စက်
ဥက္ကာ ကွန့် အိမ့်
ဥက္ကာ ဒေါသ် လာန်း
ဥက္ကာ ဆောင် မုန်း
သိင်္ဂီ ဗြ တန့်
သိင်္ဂီ ဂျမ် အိုက်
သိင်္ဂီ တက် သိင်္ဂ

## Testing/Evaluation

ဒီတခါတော့ test data ဖိုင်နဲ့ Transformer-based LM အသေးလေးကို evaluation လုပ်ခိုင်းကြည့်ရအောင်။  

In [103]:
!time python laphet.py --model_type transformer --test --model ./model/name/transformer.model \
--test_file ./data/myRoman/test_name.txt --seq_len 50 --batch_size 64 \
--embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:429: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Testing: 100%|██████████████████████████████████| 1

## BERT based Language Modeling

## Training

In [104]:
!time python laphet.py --model_type bert --train --data ./data/myRoman/train_name.txt \
--dev_file ./data/myRoman/dev_name.txt --model ./model/name/bert.model --seq_len 50 \
--epochs 10 --batch_size 32 --lr 0.0001 --embedding_method nn.Embedding

Epoch 1/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 189.67it/s]
Epoch 1, Training Loss: 0.2508
Epoch 1/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 597.03it/s]
Epoch 1, Validation Loss: 0.0435
Best model saved at ./model/name/bert.model with validation loss: 0.0435
Epoch 2/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 212.66it/s]
Epoch 2, Training Loss: 0.0290
Epoch 2/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 596.81it/s]
Epoch 2, Validation Loss: 0.0212
Best model saved at ./model/name/bert.model with validation loss: 0.0212
Epoch 3/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 212.33it/s]
Epoch 3, Training Loss: 0.0152
Epoch 3/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 604.70it/s]
Epoch 3, Validation Loss: 0.0138
Best model saved at ./model/name/bert.model with validation loss: 0.0138
Epoch 4/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 205.91it/s]
Epoch 4, Training Loss

## GPU Usage of BERT based LM Building

BERT based language model ကို training လုပ်နေစဉ်မှာ nvidia-smi command ကို သုံးပြီး GPU အသုံးပြုတဲ့ အချက်အလက်တွေကို စစ်ကြည့်တော့ အောက်ပါအတိုင်းပါ။  

In [105]:
!ls -lh ./model/name/bert*

-rw-rw-r-- 1 ye ye 3.4M Jan 27 23:14 ./model/name/bert.ftfz.model
-rw-rw-r-- 1 ye ye  24K Jan 27 23:14 ./model/name/bert.ftfz.model.vocab
-rw-rw-r-- 1 ye ye  11M Jan 28 22:56 ./model/name/bert.model
-rw-rw-r-- 1 ye ye  24K Jan 28 22:55 ./model/name/bert.model.vocab
-rw-rw-r-- 1 ye ye 3.4M Jan 27 23:20 ./model/name/bert.nofz.model
-rw-rw-r-- 1 ye ye  24K Jan 27 23:20 ./model/name/bert.nofz.model.vocab


## Text Generation

In [106]:
!time python laphet.py --model_type bert --generate --model ./model/name/bert.model \
--seq_len 50 --prompt "ရဲ" --no_of_generation 10 \
--embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: ရဲ ကျက် သောက် ဇမ် အာရ် လွှင် ဇမ် 

ဒီတခါတော့ "ကျော်" နဲ့ စတဲ့ နာမည်တွေကို generate လုပ်ခိုင်းကြည့်ရအောင်။ sequence length ကိုတော့ 5 ထားထားတယ်။  

In [107]:
!time python laphet.py --model_type bert --generate --model ./model/name/bert.model \
--seq_len 5 --prompt "ကျော်" --no_of_generation 10 --embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: ကျော် စုက္က သောက် သောက် နှော သုံး

ဒီတခါတော့ "မြင့်" နဲ့ စတဲ့ နာမည်တွေကို generate လုပ်ခိုင်းကြည့်ရအောင်။ sequence length ကိုတော့ 2 ပေးထားတယ်။  

In [108]:
!time python laphet.py --model_type bert --generate --model ./model/name/bert.model \
--seq_len 2 --prompt "မြင့်" --no_of_generation 10 --embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: မြင့် ထဲ သောက်
Generated Text 2: 

## Text Generation with Start-Word Input File

In [109]:
!time python laphet.py --model_type bert --generate --model ./model/name/bert.model \
--seq_len 2 --input ./data/myRoman/start_names.txt --no_of_generation 5 \
--output ./output/name/bert_gen_texts.txt --embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Random Prompt Generated: ကျဲရ်
Generated texts save

သတ်မှတ်ပေးထားတဲ့ (prompt အနေနဲ့ စပေးထားတဲ့) နာမည်အစ စာလုံးတွေကို အခြေခံပြီး BERT LM model က အောက်ပါအတိုင်း နာမည်အသစ်တွေကို ထုတ်ပေးနိုင်ပါတယ်။  

In [110]:
!cat ./output/name/bert_gen_texts.txt

ကျော် ဒန် ပွင့်
ကျော် ပံ သောက်
ကျော် သော့ခ် ခုံ
ကျော် ဂျော် ဟေး
ကျော် ရည် ထွေ
မ မ ဟေး သောက်
မ မ ဘောမ် ခြာ
မ မ လှန် သောက်
မ မ ချိတ် ဖွာ
မ မ ကမ္ဘာ လုံ
အေး ဘယ်လ် စက်ဖ်
အေး ခို လွှင်
အေး ကွန့် စိမ်
အေး သျှန္တီ သောက်
အေး အာရ် အိပ်
လှ လှ ရည် ဖွာ
လှ လှ ပါယ် နတ်
လှ လှ ရွိုင် ကျော
လှ လှ ဗုတ် ဟုန်
လှ လှ ဝါး ကျွံ
ကျဲရ် ယန် ထော
ကျဲရ် မြို့ သျှန်
ကျဲရ် ရွှန်း ခေတ်
ကျဲရ် စင် ခယ်
ကျဲရ် ကွတ် သောက်
မြ အေး သောင် ဗွန်
မြ အေး ပူး ကြုံ
မြ အေး ဟွမ်း အိ
မြ အေး ဒ ဆွေး
မြ အေး ရံ ရွှန်း
သ လင် စိမ်း
သ ရောင် ဘူ
သ ဘွိုင် မော
သ အိန္မာ သောက်
သ ကြေး ပေ
မောင် သွေး ရှေး
မောင် ဆုန်း မှုန်း
မောင် ကန့် ကြုံ
မောင် မွင်း လျင့်
မောင် ခွေး အိ
မြင့် မြင့် ယျန် သိဏ်း
မြင့် မြင့် ခေါင်း ဖာ
မြင့် မြင့် ဟောမ်း သောက်
မြင့် မြင့် လစ် သတ်
မြင့် မြင့် မှန် သောက်
ရွှေ ခယ် သုံး
ရွှေ ရောင်းန် သောက်
ရွှေ ဆည်း နွံ
ရွှေ ဒူး လျှန်
ရွှေ စက္ကန့် သောက်
အဂ္ဂ ဂျက်ခ် သောက်
အဂ္ဂ အွမ် သင့်
အဂ္ဂ ရွယ် ယဉ်း
အဂ္ဂ တီး တူး
အဂ္ဂ နီးလ် သောက်
ဥက္ကာ ခ မြက်
ဥက္ကာ ဂျမ် နှော
ဥက္ကာ ဆွမ့် သိင်္ခ
ဥက္ကာ လျှမ်း အူလ္လာ
ဥက္ကာ ဇင်း ဖယ်
သိင်္ဂီ ပိုက် ငိုက်
သိင်္ဂီ ထိန် မ

## Testing/Evaluation

In [111]:
!time python laphet.py --model_type bert --test --model ./model/name/bert.model \
--test_file ./data/myRoman/test_name.txt --seq_len 50 --batch_size 64 --embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:429: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Testing: 100%|██████████████████████████████████| 1

## GPT based Language Modeling

ဒီတခါတော့ ChatGPT တို့လို GPT model ကို အခြေခံတဲ့ language model အသေးတစ်ခုကို ဆောက်ကြည့်ရအောင်။  

## Training

In [112]:
!time python laphet.py --model_type gpt --train --data ./data/myRoman/train_name.txt \
--dev_file ./data/myRoman/dev_name.txt --model ./model/name/gpt.model --seq_len 50 \
--epochs 10 --batch_size 32 --lr 0.0001 --embedding_method nn.Embedding

Epoch 1/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 199.94it/s]
Epoch 1, Training Loss: 0.0689
Epoch 1/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 586.16it/s]
Epoch 1, Validation Loss: 0.0014
Best model saved at ./model/name/gpt.model with validation loss: 0.0014
Epoch 2/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 209.32it/s]
Epoch 2, Training Loss: 0.0010
Epoch 2/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 593.09it/s]
Epoch 2, Validation Loss: 0.0004
Best model saved at ./model/name/gpt.model with validation loss: 0.0004
Epoch 3/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 211.11it/s]
Epoch 3, Training Loss: 0.0004
Epoch 3/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 552.97it/s]
Epoch 3, Validation Loss: 0.0002
Best model saved at ./model/name/gpt.model with validation loss: 0.0002
Epoch 4/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 212.10it/s]
Epoch 4, Training Loss: 0

## GPU Usage of GPT-based LM Building

In [114]:
!ls -lh ./model/name/gpt.model*

-rw-rw-r-- 1 ye ye 11M Jan 28 23:01 ./model/name/gpt.model
-rw-rw-r-- 1 ye ye 24K Jan 28 23:00 ./model/name/gpt.model.vocab


## Text Generation

In [115]:
!time python laphet.py --model_type gpt --generate --model ./model/name/gpt.model \
--seq_len 50 --prompt "ရဲ" --no_of_generation 10 --embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: ရဲ ငယ် ဆင်း ဘန့် ဆပ် ဟွေ ကီး ကျ အ

ဒီတခါတော့ --prompt အေး ဆိုတာနဲ့ --seq_len 2 ပဲ ထားပြီး generate လုပ်ခိုင်းကြည့်ရအောင်။  

In [116]:
!time python laphet.py --model_type gpt --generate --model ./model/name/gpt.model \
--seq_len 2 --prompt "အေး" --no_of_generation 10 --embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: အေး ကိ ဗျက်
Generated Text 2: အေး

ဒီတစ်ခါတော့ "မြင့်" နဲ့ စတဲ့ နာမည်တွေကို generate လုပ်ခိုင်းကြည့်မယ်။  
sequence length ကို 2 ထားမယ်။ ပြီးတော့ no of generation ကိုတော့ 5 ပဲ ထားမယ်။  

In [117]:
!time python laphet.py --model_type gpt --generate --model ./model/name/gpt.model \
--seq_len 2 --prompt "မြင့်" --no_of_generation 5 --embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: မြင့် စိမ် နှင်မ်
Generated Text 

## Text Generation with Start-Word Input File

In [118]:
!time python laphet.py --model_type gpt --generate --model ./model/name/gpt.model \
--seq_len 2 --input ./data/myRoman/start_names.txt --no_of_generation 5 \
--output ./output/name/gpt_gen_texts.txt --embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Random Prompt Generated: ချို
Generated texts saved

Save လုပ်ထားတဲ့ gpt_gen_texts.txt ဖိုင်ကို ကြည့်ကြည့်ရအောင်။  

In [119]:
!cat ./output/name/gpt_gen_texts.txt

ကျော် ညင် ဆုန်
ကျော် လာဘ် ဇောင်း
ကျော် လုံး ကင်းမ်
ကျော် လတ် ရွာ
ကျော် ဆွတ် ဘု
မ မ ဇီ ဂန္တ
မ မ ဗွီ ထွန်း
မ မ မြစ် ကုံး
မ မ မြူး ဖုံး
မ မ ရွှေ ဝင်
အေး ကွဲ လှ
အေး ခူး ဆွမ်း
အေး နေ လျှမ်း
အေး မွမ်း ဇွဲ
အေး [UNK] သျှ
လှ လှ မော ရီး
လှ လှ ဆိုက် ယား
လှ လှ ဣန္ဒာ ပေါ
လှ လှ ရုံ ညိမ်း
လှ လှ တောက် ဖေါ
ချို ပိုက် မြန်
ချို ကြယ် ဆီ
ချို သော ခက်
ချို ဆွန် ချောင်
ချို တြီ မဲ့
မြ အေး ပေ ငယ်
မြ အေး စဲ စ
မြ အေး ခိ အာ
မြ အေး ခင်း စ
မြ အေး လုံး အဲ့
သ ခို ကိန်း
သ ရှိမ်း လုတ်
သ မြူး မှောင်
သ အေး ကမ္မ
သ အိန္ဓု ထူး
မောင် အယ် ဇေါင်း
မောင် စုက္က ကြင်
မောင် ဆဲ ဟို
မောင် အဲန် ကျန်း
မောင် သို့ အောင်
မြင့် မြင့် ချင်း ရော်
မြင့် မြင့် ရွယ် တာ
မြင့် မြင့် ကျုံ သင်္ခ
မြင့် မြင့် ဘူး ခဲ
မြင့် မြင့် လှေး ယိန်း
ရွှေ စိုက် ကျူး
ရွှေ မောင်း ခေါ
ရွှေ လှိုင် ချိတ်
ရွှေ ကောလ် သမ္ဘူ
ရွှေ ခေါ လေ
အဂ္ဂ ဆုန် စွယ်
အဂ္ဂ စံ ဓမ္မာ
အဂ္ဂ ဂျူး ဟင်လ်
အဂ္ဂ စေ ဂျိုး
အဂ္ဂ ဟုန် ရှု
ဥက္ကာ ကစ္စ ကျား
ဥက္ကာ စုံ အုပ်
ဥက္ကာ ကျုံ ဟူ
ဥက္ကာ ဇိ ဖောင်
ဥက္ကာ ခေါ်လ် အိုး
သိင်္ဂီ ခြူ ဆောမ်း
သိင်္ဂီ ကြိုက် လွှင်
သိင်္ဂီ ကိုင် ဖျား
သိင်္ဂီ ချမ်း ကင်
သိင်္ဂီ 

## Bash Shell Script

just for your knowledge ပါ။  
အထက်မှာ ပြခဲ့တဲ့အတိုင်း မော်ဒယ် တစ်ခုစီကို coding လုပ်ရင်းနဲ့ စမ်းရတဲ့အခါမှာ အကြိမ်ကြိမ်အခါခါ run လိုက် coding ကို ဝင်ပြင်လိုက်လုပ်ရတာမို့လို့ command line argument တွေအတိုင်းအတာ တစ်ခုထိ သတ်မှတ်ပြီးသွားတဲ့အခါမှာ shell script ရေးလိုက်ပြီး run ပြီးမှ log တွေ၊ ထွက်လာတဲ့ output တွေကို manual ဝင်စစ်ကြည့်တာမျိုး လုပ်ခဲ့ရပါတယ်။ အဲဒီလို လုပ်မှလည်း command တွေကို ရိုက်ထည့်နေရတဲ့ အချိန်သက်သာပါတယ်။  

လက်တွေ့ experiment တွေလုပ်တဲ့အခါမှာလည်း shel script ရေးတတ်ဖို့ လိုအပ်ပါတယ်။  

Laphet LM Toolkit ကို coding လုပ်နေစဉ်မှာ သုံးခဲ့တဲ့ bash shell script က အောက်ပါအတိုင်းပါ။  

In [120]:
!cat ./train_test_name.sh

#!/bin/bash

# Updated for Laphet LM Toolkit Version 0.7
# Last updated: 27 Jan 2025

# Create the output and log directories if they don't exist
mkdir -p model/name/
mkdir -p output/name/
mkdir -p log/name/

# Function to train, generate text, and test a language model
task() {
  local model_type=$1
  local model_file="./model/name/${model_type}.model"
  local output_file="./output/name/${model_type}_gen_texts.txt"
  local log_file="./log/name/${model_type}.log"
  local train_data="./data/myRoman/train_name.txt"
  local dev_data="./data/myRoman/dev_name.txt"
  local test_data="./data/myRoman/test_name.txt"
  local start_name="./data/myRoman/start_names.txt"

  {
    echo "Training ${model_type^} language model:";
    time python -u laphet.py --model_type $model_type --train --data $train_data \
      --dev_file $dev_data --model $model_file --seq_len 50 --epochs 10 --batch_size 32 \
      --lr 0.0001 --embedding_method nn.Embedding;

    echo "Text generation:";
    time python -u lap

## Running MLP, Bi-LSTM, Transformer, BERT and GPT Language Models Building

ဒီတခါတော့ Myanmar name dataset ကိုသုံးပြီး မော်ဒယ် ၅မျိုးစလုံးရဲ့ training, text generation နဲ့ testing/evaluation အကုန် အစအဆုံးကို အထက်က shell script ကိုသုံးပြီး run ကြည့်ပါမယ်။   

In [121]:
!./train_test_name.sh

Training Mlp language model:
Epoch 1/10 (Training): 100%|█████████████████| 852/852 [00:06<00:00, 131.17it/s]
Epoch 1, Training Loss: 1.3108
Epoch 1/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 511.44it/s]
Epoch 1, Validation Loss: 1.0544
Best model saved at ./model/name/mlp.model with validation loss: 1.0544
Epoch 2/10 (Training): 100%|█████████████████| 852/852 [00:06<00:00, 141.19it/s]
Epoch 2, Training Loss: 1.0530
Epoch 2/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 506.03it/s]
Epoch 2, Validation Loss: 1.0524
Best model saved at ./model/name/mlp.model with validation loss: 1.0524
Epoch 3/10 (Training): 100%|█████████████████| 852/852 [00:06<00:00, 137.38it/s]
Epoch 3, Training Loss: 1.0522
Epoch 3/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 515.66it/s]
Epoch 3, Validation Loss: 1.0520
Best model saved at ./model/name/mlp.model with validation loss: 1.0520
Epoch 4/10 (Training): 100%|█████████████████| 852/852 [00:06<00:00, 137.34it

## GPU Usage Information

လက်ရှိ ရေးထားတဲ့ Laphet LM Toolkit က Pytorch library ကို သုံးထားပါတယ်။ Neural network ကို အခြေခံတဲ့ language modeling လုပ်တာမို့လို့ GPU ပေါ်မှာ run မှ မြန်ပါလိမ့်မယ်။ တကယ်လို့ GPU မရှိရင် ဒါမှမဟုတ် GPU က စက်မှာရှိနေပေမဲ့ setup လုပ်ထားတဲ့ Python environment က အကြောင်းတခုခုကြောင့် GPU ကို ခေါ်မသုံးနိုင်ရင်တော့ CPU ပေါ်မှာပဲ Laphet က run ပေးသွားပါလိမ့်မယ်။  

အထက်က Shell script ကို run နေစဉ်မှာ GPU usage ကို တချက် တချက် စစ်ကြည့်တော့ အောက်ပါအတိုင်းပါ။ ဆောက်နေတဲ့ မော်ဒယ်အမျိုးအစား(i.e. neural network architecture) နဲ့ ဒေတာပမာဏ အပေါ်မူတည်ပြီး GPU Usage က ကွာပါလိမ့်မယ်။ 

## FYI

ဒီ notebook ကို အစကနေ အဆုံးထိ သေချာဖတ်လာတယ် ဆိုရင်တော့ NLP, AI အနည်းဆုံးတော့ မြန်မာစာကို သုံးပြီး language model ဆောက်တာနဲ့ ပတ်သက်ပြီး စိတ်ဝင်စားတယ်လို့ ယူဆပါတယ်။ 

တခု သိစေချင်တာက Laphet LM Toolkit က NLP/AI fundamental education အတွက် ရည်ရွယ်ပြီး တပတ်လောက်နဲ့ အကြမ်းပြီးအောင် ရေးထားတာမို့လို့ production အတွက် မသုံးစေချင်ပါဘူး။ Language modeling နဲ့ ပတ်သက်ပြီး ပထမဆုံး လေ့လာတဲ့သူတွေအတွက် လွယ်ကူအောင် တတ်နိုင်သမျှ ရှင်းရှင်းလေးရေးထားတာမို့ပါ။ အဲဒါတောင်မှ တကယ်တမ်း ရေးထားတဲ့ Laphet LM code အကုန် နဲ့ background theory ကို ထဲထဲဝင်ဝင် နားလည်ဖို့ ဆိုရင် ကျောင်းသားပေါ်မူတည်ပြီး သုံးလ၊ လေးလ ကြာသွားနိုင်ပါတယ်။ အနည်းဆုံးတော့ မြန်မာကျောင်းသားတွေအတွက်က မြန်မာနာမည်တွေကို သုံးပြီးတော့ မော်ဒယ်ဆောက်ပြ၊ စာကြောင်း အသစ်တွေကို generate လုပ်ပြ၊ testing/evaluation လုပ်ပြထားတာမို့လို့ ဒီ example notebook ကို အခြေခံပြီး NLP/AI သုတေသန အတွက် အရေးကြီးတဲ့ language modeling ဆိုတာကို နည်းနည်းပါးပါး တီးမိခေါက်မိရှိသွားရင်ပဲ ဝမ်းသာပါတယ်လို့။  

ရဲကျော်သူ  
LU Lab., Myanmar  
27 Jan 2025  
Email: ykt.nlp.ai@gmail.com